<a href="https://colab.research.google.com/github/SupreethRao99/DeepCAPTCHA/blob/main/TFRecordPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook contains a protoype pipeline to convert datasets comprising of images into the TFRecord format. This is done so speed up training and to use TPU's for training. 

In [ ]:
data_dir = '/tmp/English/Fnt'

from functools import partial

IMG_HEIGHT = 32
IMG_WIDTH = 32

load_split = partial(
    tf.keras.preprocessing.image_dataset_from_directory,
    data_dir,
    validation_split=0.2,
    shuffle=True,
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,
)

ds_train = load_split(subset='training')
ds_valid = load_split(subset='validation')

class_names = ds_train.class_names
print("\nClass names: {}".format(class_names))

In [ ]:
def make_example(encoded_image, label):
    image_feature = Feature(
        bytes_list=BytesList(value=[
            encoded_image,
        ]),
    )
    label_feature = Feature(
        int64_list=Int64List(value=[
            label,
        ])
    )

    features = Features(feature={
        'image': image_feature,
        'label': label_feature,
    })
    
    example = Example(features=features)
    
    return example.SerializeToString()

In [ ]:
os.mkdir('/tmp/working')
os.mkdir('/tmp/working/training')
NUM_SHARDS = 32
PATH = '/tmp/working/training/shard_{:02d}.tfrecord'

for shard in range(NUM_SHARDS):
    ds_shard = (
        ds_train_encoded
        .shard(NUM_SHARDS, shard)
        .as_numpy_iterator()
    )
    with tf.io.TFRecordWriter(path=PATH.format(shard)) as f:
        for encoded_image, label in ds_shard:
            example = make_example(encoded_image, label)
            f.write(example)

In [ ]:
!mkdir '/tmp/working/validation'

NUM_SHARDS = 8
PATH = '/tmp/working/validation/shard_{:02d}.tfrecord'

for shard in range(NUM_SHARDS):
    ds_shard = (
        ds_valid_encoded
        .shard(NUM_SHARDS, shard)
        .as_numpy_iterator()
    )
    with tf.io.TFRecordWriter(path=PATH.format(shard)) as f:
        for encoded_image, label in ds_shard:
            example = make_example(encoded_image, label)
            f.write(example)

In [ ]:
train_filenames = tf.io.gfile.glob('/tmp/working/training*.tfrec')
validation_filenames = tf.io.gfile.glob('/tmp/working/validation*.tfrec')

dataset = tf.data.TFRecordDataset(train_filenames)

In [ ]:
%cp -av "/tmp/working" "/content/drive/MyDrive/"